<a href="https://colab.research.google.com/github/arybressane/CEBD1260-BIG-DATA-ANALYTICS/blob/master/week8/recsys/collaborative_filtering/collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 3.3MB 4.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/ec/c0/55/3a28eab06b53c220015063ebbdb81213cd3dcbb72c088251ec
Successfully built scikit-surprise


In [0]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from surprise import SVD, KNNBasic, KNNBaseline
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

sns.set_style('whitegrid')

# Load the files

In [7]:
# load the movies file
url = 'https://raw.githubusercontent.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS/master/week8/recsys/collaborative_filtering/data/movielens100k/u.item'
df_movies = pd.read_csv(url, sep='|', header=None, encoding='latin')
df_movies = df_movies[[0,1]]
df_movies.columns = ['item_id', 'movie']
df_movies.head()

,item_id,movie
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [8]:
# load the ratings file
url = 'https://raw.githubusercontent.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS/master/week8/recsys/collaborative_filtering/data/movielens100k/u.data'
df_ratings = pd.read_csv(url, sep='\t', header=None)
df_ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
df_ratings.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [0]:
# load the ratings file as a dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'item_id', 'rating']], reader=reader)

# Model Training and Evaluation

In [19]:
# Split train and test
trainset, testset = train_test_split(data, test_size=0.1, shuffle=True)
# Instantiate a model and train
model = KNNBasic()
model.fit(trainset)
# Predict the test set
pred = model.test(testset)
# Evaluate the results
accuracy.rmse(pred)
accuracy.mae(pred)
pred

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9669
MAE:  0.7630


[Prediction(uid=506, iid=482, r_ui=5.0, est=4.273443602542826, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=821, iid=284, r_ui=3.0, est=3.4552128210449853, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=901, iid=1120, r_ui=4.0, est=2.8843403253014603, details={'actual_k': 15, 'was_impossible': False}),
 Prediction(uid=682, iid=721, r_ui=4.0, est=3.406317009524876, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=339, iid=447, r_ui=4.0, est=3.5707013971404424, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=882, iid=496, r_ui=5.0, est=4.487061626590552, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=929, iid=517, r_ui=5.0, est=3.920649108352353, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=512, iid=313, r_ui=3.0, est=4.539052793409814, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=870, iid=258, r_ui=4.0, est=3.743683686965085, details={'act

In [20]:
# define a cross-validation iterator
kf = KFold(n_splits=10, shuffle=True)
model = KNNBasic()
for trainset, testset in kf.split(data):
    # train and test algorithm.
    model.fit(trainset)
    predictions = model.test(testset)
    # evaluate the model
    accuracy.rmse(predictions, verbose=True)
    accuracy.mae(predictions, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9726
MAE:  0.7686
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9643
MAE:  0.7578
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9748
MAE:  0.7681
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9662
MAE:  0.7643
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9775
MAE:  0.7705
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9602
MAE:  0.7612
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9705
MAE:  0.7686
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9745
MAE:  0.7688
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9766
MAE:  0.7722
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9906
MAE:  0.7813


# Generate recommendations

In [21]:
#model = KNNBasic()
#model = KNNBaseline()
#model = SVD()
trainset = data.build_full_trainset()
model.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [22]:
# select a random user
user_id = df_ratings.sample(1)['user_id'].values[0]
print('selected user', user_id)

# select the past votes for the random user
df_user = df_ratings.copy()[df_ratings['user_id']==user_id]
print(df_user.merge(df_movies, how='left', on='item_id')[['item_id','rating','movie']].sort_values(by='rating', ascending=False))

# select the potential recommendations
not_voted = set(df_movies['item_id'].values)-set(df_user['item_id'].values)
print('number of potential recommendations', len(not_voted))

# calculate the prediction
pred = []
for i in list(not_voted):
    pred.append((user_id, i, model.predict(str(user_id), str(i))))
    
# show the list of recommendations
recommendations = []
for uid, iid, est in pred:
    recommendations.append([uid, iid, est[3]])
df_recommendations = pd.DataFrame(recommendations)
df_recommendations.columns = ['user_id', 'item_id', 'prediction']
df_recommendations['item_id'] = df_recommendations['item_id'].astype(float)
df_recommendations = df_recommendations.sort_values('prediction', ascending=False)
df_recommendations.merge(df_movies, how='left', on='item_id')

selected user 887
     item_id  rating                                          movie
85       172       5                Empire Strikes Back, The (1980)
60       176       5                                  Aliens (1986)
69        28       5                               Apollo 13 (1995)
68       294       5                               Liar Liar (1997)
66        47       5                                 Ed Wood (1994)
132      378       5                  Miracle on 34th Street (1994)
64       969       5    Winnie the Pooh and the Blustery Day (1968)
104      118       5                                 Twister (1996)
105      151       5   Willy Wonka and the Chocolate Factory (1971)
61       421       5  William Shakespeare's Romeo and Juliet (1996)
152     1060       5            Adventures of Pinocchio, The (1996)
151      720       5                            First Knight (1995)
138     1084       5                   Anne Frank Remembered (1995)
107      128       5          

,user_id,item_id,prediction,movie
0,887,2.0,3.52986,GoldenEye (1995)
1,887,1167.0,3.52986,"Sum of Us, The (1994)"
2,887,1176.0,3.52986,Welcome To Sarajevo (1997)
3,887,1175.0,3.52986,Hugo Pool (1997)
4,887,1174.0,3.52986,Caught (1996)
5,887,1173.0,3.52986,Bliss (1997)
6,887,1172.0,3.52986,"Women, The (1939)"
7,887,1171.0,3.52986,Wild Reeds (1994)
8,887,1170.0,3.52986,Spanking the Monkey (1994)
9,887,1169.0,3.52986,Fresh (1994)
